### Step 2: merge all .csv files into a one-year dataframe

loop through the lob_caps directory, forming one time-sorted dataframe, with all CAPS files. These files captured sample bid and ask capitalization, and respective bid and ask volumes.

In [3]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [38]:
!pip3 install matplotlib
!pip3 install altair
!pip3 install sklearn

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post10-py3-none-any.whl size=2958 sha256=0e0fc7d67b2ad10d716db5d5baea8d6a247a486bdd53004020071b4c0d6750c0
  Stored in directory: /Users/stefanbund/Library/Caches/pip/wheels/59/9c/1b/6be8fc606a34b098ddb96d3d5e07fb21d49599055fa0c0fe87
Successfully built sklearn


In [2]:
import altair as alt
import pandas as pd
import os
import numpy as np

In [4]:
# !mv $(find . -type d -name "lob_caps" -exec grep -q MATCH {} \; -print0 | xargs -0 echo) backup_match/

In [4]:
#https://stackoverflow.com/a/21232849 model 
def getCAPSByDateAndType(type):  #returns a dict, date + df caps for that date, then extended date and time
                                # print("for type, ", type)  ./lob_caps/
    ret = []
    for root, dirs, files in os.walk("../lob_caps"): #core/gh-code/grus-code/ver2-pctChangeDriven/lob_caps
        for filename in files:
            if type in filename:
                # print("CAPS file, ", filename) #mac, do find . -name ._\* -delete
                ret.append(filename)
    return ret

csvFileList = getCAPSByDateAndType("CAPS") #iterate this array to dip into each csv, later on
li = []                         #form the endFrame / global data frame around this array
for filename in csvFileList:
    csv = "../lob_caps/" + filename
    # print(csv)
    df = pd.read_csv(csv, index_col=None, header=0)
    li.append(df)

capsFrame = pd.concat(li, axis=0, ignore_index=True) #end frame contains all data
capsFrame.sort_values(by=['time'], ascending=True)   #sorted by time into one time series
print("for new df: ", capsFrame.shape[0])
start = capsFrame["time"].min()
end = capsFrame["time"].max()
print("start: ", start, " end: ", end)
print(capsFrame.columns)

for new df:  298660
start:  1660221600292.0  end:  1693078943553.0
Index(['bc', 'ac', 'tbv', 'tav', 'time', 'mp', 'minBid'], dtype='object')


In [5]:
capsFrame

,bc,ac,tbv,tav,time,mp,minBid
0,2435138.24,10557430.61,493817.73,244833.13,1.672418e+12,10.78,10.74
1,2430127.12,10557235.51,493352.24,244815.55,1.672418e+12,10.78,10.74
2,2424788.47,10557236.44,492823.44,244815.55,1.672418e+12,10.78,10.74
3,2423662.55,10557235.85,492718.37,244815.55,1.672418e+12,10.78,10.74
4,2424625.42,10557236.44,492808.51,244815.55,1.672418e+12,10.78,10.74
...,...,...,...,...,...,...,...
298655,2493120.60,10145161.66,422173.39,245978.60,1.684076e+12,15.05,14.79
298656,2496909.84,10148152.98,422425.45,246176.99,1.684076e+12,15.05,14.79
298657,2467602.13,10195139.78,420251.33,249169.43,1.684077e+12,15.12,14.79
298658,2478721.33,10191311.21,420969.77,248923.39,1.684078e+12,15.12,14.79


## schema for capitalization data

loads the csv files, as acquired from coinbase

In [6]:
capsFrame.head(2) #shows the basic data collection via coinbase, these are aggregated values, collected several x a minute

,bc,ac,tbv,tav,time,mp,minBid
0,2435138.24,10557430.61,493817.73,244833.13,1.672418e+12,10.78,10.74
1,2430127.12,10557235.51,493352.24,244815.55,1.672418e+12,10.78,10.74


### imputation

In [7]:
# impute missing values with last non-null value
capsFrame['bc'] = capsFrame['bc'].fillna(method='ffill')
capsFrame['ac'] = capsFrame['ac'].fillna(method='ffill')
capsFrame['tbv'] = capsFrame['tbv'].fillna(method='ffill')
capsFrame['tav'] = capsFrame['tav'].fillna(method='ffill')
capsFrame['mp'] = capsFrame['mp'].fillna(method='ffill')
capsFrame['minBid'] = capsFrame['minBid'].fillna(method='ffill')


In [8]:
capsFrame.to_csv("global_data.csv")

## Discover precursor and surge episodes

the goal of the data prep is to discover periods of continuous, positive momentum. These are **surges**. 

The periods preceding surges are, for the sake of the experiment, **precursors**. They are detected as periods of discontinuous positive momentum, or negative momentum. 

A ten-row window is used to calculate positive or negative momentum. A percent **change** is calculated for the ten row subsample.

## regularization of critical features
get percent change as basis for comprehending LOB

create new columns which depict the momentum of one row versus the next, in terms of price , capitalization and volume

In [13]:
# Load your time series data into a pandas dataframe
# consider cahnging this approach because it doesnt actually check in between values

caps_df = capsFrame   
lookback_period = 10 # in rows
caps_df['change'] = caps_df['mp'].pct_change(periods=lookback_period)
caps_df['bc_change'] = caps_df['bc'].pct_change(periods=lookback_period)
caps_df['ac_change'] = caps_df['ac'].pct_change(periods=lookback_period)
caps_df['tav_change'] = caps_df['tav'].pct_change(periods=lookback_period)
caps_df['tbv_change'] = caps_df['tbv'].pct_change(periods=lookback_period)
## key components: bc_change, ac_change, tav_change, tbv_change, change
# caps_df.sample
print(caps_df.shape[0], caps_df.columns)# Calculate the returns of your asset over a fixed lookback period

298660 Index(['bc', 'ac', 'tbv', 'tav', 'time', 'mp', 'minBid', 'change', 'bc_change',
       'ac_change', 'tav_change', 'tbv_change'],
      dtype='object')


###  establish benchmarks for percent change

the mean of change represents the average rate of change between LOB samples. This is used to determine whether the change between rows is significant or not. 

In [14]:
#for period, average or mean change metric. this changes with window size
meanChange = round(caps_df['change'].mean(),8)
meanChange

0.00047286

## data mining: sequence discovery
define precursors from surges, prepare the data with this sequence: 

precursor -> surge

prepare to cluster every precursor, by the sequential, resultant surge. Do not assume causality, but rather preoccurance.

use the threshold, mean change as tool to separate precursor from surges, where surges represent periods of positive momentum above threshold.

This step defines the data schema for the remainder of the process, where key statistics are defined for precursors and surges.

In [15]:
# identify units of 10 rows where the percent change is greater or less than the threshold
### key components: bc_change, ac_change, tav_change, tbv_change, change
threshold = meanChange
surges = []
precursors = []
for i in range(0,len(caps_df),10):
    if caps_df.iloc[i:i+10]['change'].mean() >= threshold:
        surges.append({'time': caps_df.iloc[i]['time'],
                       's_MP': caps_df.iloc[i]['mp'],
                       'change': caps_df.iloc[i:i+10]['change'].mean(),
                       'type':'surge'})  #['bc', 'ac', 'tbv', 'tav', 'time', 'mp', 'minBid', 'change']
    else:
        precursors.append({'time': caps_df.iloc[i]['time'],
                           'p_MP': caps_df.iloc[i]['mp'],
                           'change': caps_df.iloc[i:i+10]['change'].mean(),
                            'type':'precursor',
                            'precursor_buy_cap_pct_change':caps_df.iloc[i]['bc_change'], 
                            'precursor_ask_cap_pct_change':caps_df.iloc[i]['ac_change'],
                            'precursor_bid_vol_pct_change':caps_df.iloc[i]['tbv_change'],
                            'precursor_ask_vol_pct_change':caps_df.iloc[i]['tav_change']
                            })  

In [16]:
#for item in surges[:2]:
    #print(item)

In [17]:
#for item in precursors:
    #print(item)

## prepprocess: merge precursors and surges into time series

a dataframe of sequences, **sequence_df** is created by concatenating both buckets, and sorting by time. This will create a time series of surge and precursor periods, as defined by: 

* 10 window percent change values
* contiguity: these precursor and surges are next to each other and thus have a length or duration of momentum.

In [18]:
surges_df = pd.DataFrame(surges)
precursors_df = pd.DataFrame(precursors)
sequence_df = pd.concat([surges_df, precursors_df]).sort_values(by=['time'], ascending=True)

In [19]:
sequence_df.index

Index([ 7861,  7862, 13991, 13992, 13993, 13994, 13995, 13996,  7863, 13997,
       ...
        8519,  8520,  8521,  8522,  8523,  4671,  8524,  8525,  4672,  4673],
      dtype='int64', length=29866)

### view the aligned, continuous time series of precursors and surges

view the final abstraction: sets of precursor periods, next to surges, in a linear time series. Precursors effectively precede surges on a linear time series.

In [20]:
# for index, row in sequence_df.iterrows():
#     print(row['surge'], row['precursor'])
sequence_df['type'].head(40)

7861         surge
7862         surge
13991    precursor
13992    precursor
13993    precursor
13994    precursor
13995    precursor
13996    precursor
7863         surge
13997    precursor
7864         surge
7865         surge
13998    precursor
13999    precursor
14000    precursor
7866         surge
14001    precursor
14002    precursor
7867         surge
7868         surge
14003    precursor
14004    precursor
14005    precursor
14006    precursor
14007    precursor
14008    precursor
14009    precursor
7869         surge
7870         surge
14010    precursor
14011    precursor
7871         surge
7872         surge
7873         surge
7874         surge
14012    precursor
14013    precursor
14014    precursor
14015    precursor
7875         surge
Name: type, dtype: object

In [21]:
# sequence_df.head(45)

## visualize proof of algorithmic accuracy

this chart will plot the price time series, with an area of precursor and surge, as proof of our algorithmic accuracy.

In [22]:
subset = sequence_df[:4999]
line = alt.Chart(subset).mark_line(color='green').encode(
    x='time',
    y='s_MP'
)

s_bar = alt.Chart(subset).mark_bar().encode(
    x='time',
    y='s_MP',
    color='type:N'
)

p_bar = alt.Chart(subset).mark_bar().encode(
    x='time',
    y='p_MP',
    color='type:N'
)

chart = (s_bar + p_bar + line).properties(width=600, height=500)
chart.title = 'Data Mining Accuracy, Surge vs Precursor Sequence'
subtitle = 'Precursors are contiguous periods where percentage rate of growth is less than threshold'
chart.properties(title=alt.TitleParams(text=[chart.title, subtitle], baseline='bottom', orient='top', anchor='start', fontSize=14))
chart.interactive()

alt.LayerChart(...)

In [23]:
sequence_df.columns

Index(['time', 's_MP', 'change', 'type', 'p_MP',
       'precursor_buy_cap_pct_change', 'precursor_ask_cap_pct_change',
       'precursor_bid_vol_pct_change', 'precursor_ask_vol_pct_change'],
      dtype='object')

### data mining 2: information gain, create new features

Perform information gain on grouped precursors and surges

define the **sum change**, or total change per continuous episode (precursor or surge). 

define the **length** of each episode. 

define the height of the surge, how high did the continuous positive momentum reach?

define the size (area) of the surge, as a triangular area (height times length), as **surge_area**

Create one line to describe a precursor or search and it's related order book statistics

In [24]:
sequence_df['group'] = (sequence_df['type'] != sequence_df['type'].shift(1)).cumsum()
columns_to_transform = [
    'precursor_buy_cap_pct_change',
    'precursor_ask_cap_pct_change',
    'precursor_bid_vol_pct_change',
    'precursor_ask_vol_pct_change'
]

for col in columns_to_transform:
    sequence_df[col] = sequence_df.groupby('group')[col].transform(lambda x: x.sum() if not x.isna().all() else np.nan)

In [25]:
#### imputation

In [26]:
# # impute missing values with last non-null value DONE PRIOR, NOW AT START
sequence_df['s_MP'] = sequence_df['s_MP'].fillna(method='ffill')
sequence_df['p_MP'] = sequence_df['p_MP'].fillna(method='ffill')
sequence_df['precursor_buy_cap_pct_change'] = sequence_df['precursor_buy_cap_pct_change'].fillna(method='ffill')
sequence_df['precursor_ask_cap_pct_change'] = sequence_df['precursor_ask_cap_pct_change'].fillna(method='ffill')
sequence_df['precursor_bid_vol_pct_change'] = sequence_df['precursor_bid_vol_pct_change'].fillna(method='ffill')
sequence_df['precursor_ask_vol_pct_change'] = sequence_df['precursor_ask_vol_pct_change'].fillna(method='ffill')

In [27]:
#sequence_df['group'] = (sequence_df['type'] != sequence_df['type'].shift(1)).cumsum()

In [28]:
sequence_df['length'] = sequence_df.groupby(['type', 'group'])['group'].transform('count')

print(sequence_df.shape[0])
sequence_df['sum_change'] = sequence_df.groupby(['type', 'group'])['change'].transform('sum')

sequence_df['max_surge_mp'] = sequence_df.groupby(['type', 'group'])['s_MP'].transform('max')
sequence_df['min_surge_mp'] = sequence_df.groupby(['type', 'group'])['s_MP'].transform('min')

sequence_df['max_precursor_mp'] = sequence_df.groupby(['type', 'group'])['p_MP'].transform('max')
sequence_df['min_precursor_mp'] = sequence_df.groupby(['type', 'group'])['p_MP'].transform('min')

sequence_df['area']  = sequence_df.apply(lambda row: row['length'] * row['sum_change'], axis=1)

sequence_df.loc[sequence_df['type'] == 'surge', 'surge_area'] = sequence_df.loc[sequence_df['type'] == 'surge', 'area']

sequence_df['surge_targets_met_pct']  = sequence_df.apply(lambda group: ((group['max_precursor_mp']-group['max_surge_mp'])/group['max_surge_mp']  ) *100, axis=1)

29866


In [29]:
# define a custom function to calculate the percentage by which max_surge_mp exceeds max_precursor_mp
'''for a pandas dataframe wth attributes ['group', 'time', 's_MP', 'change', 'type', 'length', 'sum_change',
       'max_surge_mp', 'min_surge_mp', 'area', 'surge_area', 'group', 'time',
       'change', 'type', 'p_MP', 'precursor_buy_cap_pct_change',
       'precursor_ask_cap_pct_change', 'precursor_bid_vol_pct_change',
       'precursor_ask_vol_pct_change', 'length', 'sum_change',
       'max_precursor_mp', 'min_precursor_mp', 'area'] 
       group by type, group then create  
       a new column 'surge_targets_met_pct' which equals the percentage 
       by which the max_surge_mp exceeds the max_precursor_mp'''


sequence_df.columns
print(sequence_df.shape[0])

29866


In [30]:
sequence_df.head(30)

,time,s_MP,change,type,p_MP,precursor_buy_cap_pct_change,precursor_ask_cap_pct_change,precursor_bid_vol_pct_change,precursor_ask_vol_pct_change,group,length,sum_change,max_surge_mp,min_surge_mp,max_precursor_mp,min_precursor_mp,area,surge_area,surge_targets_met_pct
7861,1.660222e+12,29.95,0.787720,surge,NaN,NaN,NaN,NaN,NaN,1,2,0.788422,29.99,29.95,NaN,NaN,1.576844,1.576844,NaN
7862,1.660222e+12,29.99,0.000701,surge,NaN,NaN,NaN,NaN,NaN,1,2,0.788422,29.99,29.95,NaN,NaN,1.576844,1.576844,NaN
13991,1.660222e+12,29.99,-0.000633,precursor,29.94,-0.000798,0.000046,-0.000173,0.006228,2,6,-0.004876,29.99,29.99,29.97,29.84,-0.029257,NaN,-0.066689
13992,1.660222e+12,29.99,-0.001167,precursor,29.97,-0.000798,0.000046,-0.000173,0.006228,2,6,-0.004876,29.99,29.99,29.97,29.84,-0.029257,NaN,-0.066689
13993,1.660222e+12,29.99,-0.001169,precursor,29.94,-0.000798,0.000046,-0.000173,0.006228,2,6,-0.004876,29.99,29.99,29.97,29.84,-0.029257,NaN,-0.066689
13994,1.660222e+12,29.99,-0.000668,precursor,29.88,-0.000798,0.000046,-0.000173,0.006228,2,6,-0.004876,29.99,29.99,29.97,29.84,-0.029257,NaN,-0.066689
13995,1.660222e+12,29.99,-0.000402,precursor,29.90,-0.000798,0.000046,-0.000173,0.006228,2,6,-0.004876,29.99,29.99,29.97,29.84,-0.029257,NaN,-0.066689
13996,1.660222e+12,29.99,-0.000837,precursor,29.84,-0.000798,0.000046,-0.000173,0.006228,2,6,-0.004876,29.99,29.99,29.97,29.84,-0.029257,NaN,-0.066689
7863,1.660222e+12,29.84,0.001039,surge,29.84,-0.000798,0.000046,-0.000173,0.006228,3,1,0.001039,29.84,29.84,29.84,29.84,0.001039,0.001039,0.000000
13997,1.660222e+12,29.84,-0.000134,precursor,29.87,-0.003129,-0.000032,-0.001081,-0.004203,4,1,-0.000134,29.84,29.84,29.87,29.87,-0.000134,NaN,0.100536


## data mining 3: form final sequences by statistical weight

Critical group by unique identifier

In [31]:
unique_df = sequence_df.groupby('group').first().reset_index()
# print(unique_df)

In [32]:
unique_df.head(20)

,group,time,s_MP,change,type,p_MP,precursor_buy_cap_pct_change,precursor_ask_cap_pct_change,precursor_bid_vol_pct_change,precursor_ask_vol_pct_change,length,sum_change,max_surge_mp,min_surge_mp,max_precursor_mp,min_precursor_mp,area,surge_area,surge_targets_met_pct
0,1,1.660222e+12,29.95,0.787720,surge,NaN,NaN,NaN,NaN,NaN,2,0.788422,29.99,29.95,NaN,NaN,1.576844,1.576844,NaN
1,2,1.660222e+12,29.99,-0.000633,precursor,29.94,-0.000798,0.000046,-0.000173,0.006228,6,-0.004876,29.99,29.99,29.97,29.84,-0.029257,NaN,-0.066689
2,3,1.660222e+12,29.84,0.001039,surge,29.84,-0.000798,0.000046,-0.000173,0.006228,1,0.001039,29.84,29.84,29.84,29.84,0.001039,0.001039,0.000000
3,4,1.660222e+12,29.84,-0.000134,precursor,29.87,-0.003129,-0.000032,-0.001081,-0.004203,1,-0.000134,29.84,29.84,29.87,29.87,-0.000134,NaN,0.100536
4,5,1.660222e+12,29.91,0.001273,surge,29.87,-0.003129,-0.000032,-0.001081,-0.004203,2,0.001875,29.92,29.91,29.87,29.87,0.003749,0.003749,-0.167112
5,6,1.660222e+12,29.92,-0.000701,precursor,29.90,0.000440,-0.000007,0.000294,-0.000717,3,-0.004013,29.92,29.92,29.90,29.83,-0.012040,NaN,-0.066845
6,7,1.660222e+12,29.83,0.002148,surge,29.83,0.000440,-0.000007,0.000294,-0.000717,1,0.002148,29.83,29.83,29.83,29.83,0.002148,0.002148,0.000000
7,8,1.660222e+12,29.83,0.000369,precursor,29.89,-0.003818,0.000013,-0.001505,0.001104,2,-0.000300,29.83,29.83,29.90,29.89,-0.000601,NaN,0.234663
8,9,1.660223e+12,29.90,0.001038,surge,29.90,-0.003818,0.000013,-0.001505,0.001104,2,0.002209,29.92,29.90,29.90,29.90,0.004419,0.004419,-0.066845
9,10,1.660223e+12,29.92,-0.001637,precursor,29.90,0.009044,0.000025,0.004399,0.007863,7,-0.013463,29.92,29.92,29.90,29.53,-0.094238,NaN,-0.066845


#### Merge even and odd Rows to form the final sequences

Even rows contain surge, and odd rows contain precursors. **When you merge them, you form a sequence of precursor, and surge.**

Each row will contain a continuous **precursor->surge** sequence.

In [33]:
# needs to start with a precursor removes the first surge
unique_df = unique_df.iloc[1:]
even_df = unique_df.iloc[::2].reset_index(drop=True)
odd_df = unique_df.iloc[1::2].reset_index(drop=True)

merged_df = pd.concat([even_df, odd_df], axis=1)

# print(merged_df)

In [34]:
merged_df[:10]

,group,time,s_MP,change,type,p_MP,precursor_buy_cap_pct_change,precursor_ask_cap_pct_change,precursor_bid_vol_pct_change,precursor_ask_vol_pct_change,...,precursor_ask_vol_pct_change,length,sum_change,max_surge_mp,min_surge_mp,max_precursor_mp,min_precursor_mp,area,surge_area,surge_targets_met_pct
0,2,1.660222e+12,29.99,-0.000633,precursor,29.94,-0.000798,0.000046,-0.000173,0.006228,...,0.006228,1,0.001039,29.84,29.84,29.84,29.84,0.001039,0.001039,0.000000
1,4,1.660222e+12,29.84,-0.000134,precursor,29.87,-0.003129,-0.000032,-0.001081,-0.004203,...,-0.004203,2,0.001875,29.92,29.91,29.87,29.87,0.003749,0.003749,-0.167112
2,6,1.660222e+12,29.92,-0.000701,precursor,29.90,0.000440,-0.000007,0.000294,-0.000717,...,-0.000717,1,0.002148,29.83,29.83,29.83,29.83,0.002148,0.002148,0.000000
3,8,1.660222e+12,29.83,0.000369,precursor,29.89,-0.003818,0.000013,-0.001505,0.001104,...,0.001104,2,0.002209,29.92,29.90,29.90,29.90,0.004419,0.004419,-0.066845
4,10,1.660223e+12,29.92,-0.001637,precursor,29.90,0.009044,0.000025,0.004399,0.007863,...,0.007863,2,0.004468,29.67,29.59,29.53,29.53,0.008936,0.008936,-0.471857
5,12,1.660225e+12,29.67,-0.001989,precursor,29.68,0.004413,0.000151,0.001655,0.020194,...,0.020194,4,0.008090,29.80,29.68,29.52,29.52,0.032358,0.032358,-0.939597
6,14,1.660227e+12,29.80,-0.003916,precursor,29.90,-0.017945,-0.000209,-0.002911,-0.016354,...,-0.016354,1,0.006742,29.29,29.29,29.06,29.06,0.006742,0.006742,-0.785251
7,16,1.660230e+12,29.29,-0.005292,precursor,29.28,0.013487,0.000042,0.005258,0.010930,...,0.010930,1,0.001897,29.10,29.10,29.05,29.05,0.001897,0.001897,-0.171821
8,18,1.660232e+12,29.10,-0.002128,precursor,29.14,-0.008301,0.000101,-0.001793,0.015760,...,0.015760,2,0.003292,28.92,28.89,28.83,28.83,0.006585,0.006585,-0.311203
9,20,1.660236e+12,28.92,-0.001968,precursor,28.87,-0.005147,-0.000067,-0.001574,-0.008874,...,-0.008874,1,0.004051,28.99,28.99,28.87,28.87,0.004051,0.004051,-0.413936


In [35]:
nan_cols = merged_df.dropna(axis=1, how='all')
nan_cols.head()

,group,time,s_MP,change,type,p_MP,precursor_buy_cap_pct_change,precursor_ask_cap_pct_change,precursor_bid_vol_pct_change,precursor_ask_vol_pct_change,...,precursor_ask_vol_pct_change,length,sum_change,max_surge_mp,min_surge_mp,max_precursor_mp,min_precursor_mp,area,surge_area,surge_targets_met_pct
0,2,1.660222e+12,29.99,-0.000633,precursor,29.94,-0.000798,0.000046,-0.000173,0.006228,...,0.006228,1,0.001039,29.84,29.84,29.84,29.84,0.001039,0.001039,0.000000
1,4,1.660222e+12,29.84,-0.000134,precursor,29.87,-0.003129,-0.000032,-0.001081,-0.004203,...,-0.004203,2,0.001875,29.92,29.91,29.87,29.87,0.003749,0.003749,-0.167112
2,6,1.660222e+12,29.92,-0.000701,precursor,29.90,0.000440,-0.000007,0.000294,-0.000717,...,-0.000717,1,0.002148,29.83,29.83,29.83,29.83,0.002148,0.002148,0.000000
3,8,1.660222e+12,29.83,0.000369,precursor,29.89,-0.003818,0.000013,-0.001505,0.001104,...,0.001104,2,0.002209,29.92,29.90,29.90,29.90,0.004419,0.004419,-0.066845
4,10,1.660223e+12,29.92,-0.001637,precursor,29.90,0.009044,0.000025,0.004399,0.007863,...,0.007863,2,0.004468,29.67,29.59,29.53,29.53,0.008936,0.008936,-0.471857


In [36]:
nan_cols.columns

Index(['group', 'time', 's_MP', 'change', 'type', 'p_MP',
       'precursor_buy_cap_pct_change', 'precursor_ask_cap_pct_change',
       'precursor_bid_vol_pct_change', 'precursor_ask_vol_pct_change',
       'length', 'sum_change', 'max_surge_mp', 'min_surge_mp',
       'max_precursor_mp', 'min_precursor_mp', 'area', 'surge_targets_met_pct',
       'group', 'time', 's_MP', 'change', 'type', 'p_MP',
       'precursor_buy_cap_pct_change', 'precursor_ask_cap_pct_change',
       'precursor_bid_vol_pct_change', 'precursor_ask_vol_pct_change',
       'length', 'sum_change', 'max_surge_mp', 'min_surge_mp',
       'max_precursor_mp', 'min_precursor_mp', 'area', 'surge_area',
       'surge_targets_met_pct'],
      dtype='object')

### Write to CSV: step one, pipeline
Label to use is surge_targets_met_pct

In [37]:
nan_cols.to_csv('pipeline1.csv', index=False)